<div style="text-align:center">
    <h2> CS3100 - Lecture 08 - Tue, Aug 17, 11am </h2>
    <h1> More on Functions </h1>
</div>

## Recap

CS3100 so far

* Expressions, Values, Definitions: if, let
* Static and Dynamic Semantics
* Basics of Functions

Today

* More on Functions

## Functions - the main concepts learned so far

* Functions are **values** 
    - Functions can **take** functions as arguments
    - Functions can **evaluate** to functions as the result
* Applying functions.
* Using Let definition to give names to anonymous functions.
* Multi-argument functions do not exist. Synactic sugar.

## Function application

The syntax is 

```ocaml
e0 e1 ... en
```
* No parentheses necessary

## Two examples to recap

Read

```ocaml
(fun x y -> x + y)
```

as 

```ocaml
fun x -> fun y -> x+y
```

Read

```ocaml
(fun x y -> x + y) 1 2
```

as 

```ocaml
(((fun x -> fun y -> x + y) 1) 2)
```

## Function types

In [23]:
fun (x:int) (y:int) (z:int) -> x+y

- : int -> int -> int -> int = <fun>


In [24]:
fun x y z -> x

- : 'a -> 'b -> 'c -> 'a = <fun>


In [27]:
let f() = 1 + 2 * 3

val f : unit -> int = <fun>


## Function types

A function of type

```ocaml
int -> int -> int -> int
```

can be read as 

```ocaml
int -> (int -> int -> int)
```

Take 1 argument and return a function!

## Function types are right associative

```ocaml
int -> int -> int
```

is equivalent to

```ocaml
int -> (int -> int)
```

## Partial Application

```ocaml
(fun x y z -> x + y + z) 1
```

returns a function 

```ocaml
(fun y z -> 1 + y + z)
```

## Function definition

In [28]:
let add x y = x + y

val add : int -> int -> int = <fun>


In [ ]:
let add = fun x -> fun y -> x + y

In [29]:
add 5 10

- : int = 15


## We can annotate types

In [ ]:
let add : int -> (int -> int) = 
  fun x -> fun y -> x + y

## Partial Application

```ocaml
(fun x y z -> x + y + z) 1
```

returns a function 

```ocaml
(fun y z -> 1 + y + z)
```

In [30]:
let foo = (fun x y z -> x + y + z) 1

val foo : int -> int -> int = <fun>


In [31]:
foo 2 3

- : int = 6


## Partial Application

A more useful partial application example is defining `succ` and `pred` functions from `add`.

In [32]:
let succ = add 1
let pred = add (-1)

val succ : int -> int = <fun>


val pred : int -> int = <fun>


In [33]:
succ 10

- : int = 11


In [34]:
pred 10

- : int = 9


## Scope for function definitions

We can use let expressions to define scope for function definitions

In [35]:
let myFunction = (fun y -> fun x -> x-y) 5 in myFunction 10

- : int = 5


In [ ]:
succ 10

In [ ]:
pred 10

## Recursive Functions

Recursive functions can call themselves. The syntax for recursive function definition is:

```ocaml
let rec foo x = ...
```

Notice the `rec` key word.

## Recursive Functions

In [36]:
let rec findsum n = 
  if n <= 0 then 0
  else n + findsum (n-1)

val findsum : int -> int = <fun>


In [37]:
findsum 5

- : int = 15


## Tracing functions in Jupyter

Jupyter (really the ocaml top-level behind the scenes) provides support for tracing the execution of functions.

In [41]:
#trace findsum;;
findsum 3;;

- : int = 6


findsum is already traced (under the name findsum).
findsum <-- 3
findsum <-- 2
findsum <-- 1
findsum <-- 0
findsum --> 0
findsum --> 1
findsum --> 3
findsum --> 6


In [38]:
#trace findsum;;

findsum is now traced.


In [42]:
#untrace findsum

findsum is no longer traced.


## Recursive Fibonacci number

Implement a recursive function that computes the nth fibonacci number. The fibonacci sequence is `[0;1;1;2;3;5;8;...]`.

In [ ]:
let rec fib n = 
  if n = 0 then 0
  else if n = 1 then 1
  else fib (n - 2) + fib (n - 1)

In [44]:
assert (fib 10 = 56)

error: runtime_error

In OCaml, the expression `assert e` raises a fatal error if `e` evaluates to `false`. If `e` is `true`, then `assert e` evaluates to `true`.

## Mutually recursive functions

Mutually recursive functions are a set of functions that can call each other. They are defined using

```
let rec f1 v1 v2 ... vn =
  ...
and f2 v1 v2 ... vn =
   ...
...
and fn v1 v2 ... vn =
   ...
```

where the functions `f1` to `fn` can call each other. 

In [45]:
let rec even n =
  if n = 0 then true
  else odd (n-1)
  
and odd n = 
  if n = 0 then false 
  else even (n-1)

val even : int -> bool = <fun>
val odd : int -> bool = <fun>


## Recursing too deep

Let's invoke `findsum` with larger numbers. 

In [46]:
findsum 1000000

error: runtime_error

## Stack buildup

```ocaml
let rec sum_of_first_n n = 
  if n <= 0 then 0
  else n + sum_of_first_n (n-1)
```

Some work `"+ n"` left to do after the recursive call returns. This builds up stack frames.

## Stack buildup

For `findsum 5`:

* `5 + (findsum 4)`
* `4 + (findsum 3)`
* `3 + (findsum 2)`
* `2 + (findsum 1)`
* `1 + (findsum 0)`
* the number `n` kept in stack frame.
* This causes stackoverflow for small values itself.


## Tail recursion

Rewrite the function such that the recursive call is the last thing that the function does:

In [ ]:
let rec findsum_tailrec acc n = 
  if n <= 0 then acc
  else findsum_tailrec (n + acc) (n-1)
  
let findsum_tailrec n = findsum_tailrec 0 n

In [ ]:
findsum_tailrec 1000000000

## Tail recursion

```ocaml
let rec sum_of_first_n_tailrec n acc = 
  if n <= 0 then acc
  else sum_of_first_n_tailrec (n-1) (n + acc)
```

* No work left to do when the recursive call returns except return result to caller.
* OCaml compiler does **tail call optimisation** that pops current call frame before invoking recursive call. 
  + No stack buildup => equivalent to writing a tight loop. 